In [ ]:
import copy

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pooch

from fair import FAIR
from fair.io import read_properties
from fair.interface import initialise, fill

In [ ]:
ar6_forcing_file = pooch.retrieve(
    url="doi:10.5281/zenodo.5705391/table_A3.3_historical_ERF_1750-2019_best_estimate.csv",
    known_hash="md5:f64915777d2971c5eb5d96a432f45c48",
)

In [ ]:
ar6_forcing_df = pd.read_csv(ar6_forcing_file, index_col=0)

In [ ]:
pd.options.display.max_columns = 50
fair_params_df = pd.read_csv('../data/fair2.1-parameters/ar6_calibration_ebm3.csv', index_col=0)
fair_params_df

In [ ]:
volcanic_forcing = np.zeros(351)
volcanic_forcing[:270] = ar6_forcing_df['volcanic'].values
volcanic_forcing[269:281] = np.linspace(1, 0, 12) * volcanic_forcing[269]

In [ ]:
solar_forcing=np.zeros(351)
solar_forcing_df = pd.read_csv('../data/forcing/solar_erf.csv', index_col=0)
solar_forcing = solar_forcing_df['solar_erf'].values[:351]

trend_shape = np.ones(351)
trend_shape[:271] = np.linspace(0, 1, 271)

In [ ]:
f = FAIR()
f.define_time(1750, 2100, 1)
f.define_scenarios(['ssp126'])
species, properties = read_properties()
f.define_species(species, properties)
f.define_configs([fair_params_df.index[0]])
f.allocate()
f.fill_from_rcmip()
# Volcanic forcing
fill(f.forcing, volcanic_forcing[:, None, None] * fair_params_df.loc[fair_params_df.index[0], 'scale Volcanic'], specie='Volcanic')

# Solar forcing
trend_shape = np.ones(351)
trend_shape[:271] = np.linspace(0, 1, 271)

fill(f.forcing, 
     solar_forcing[:, None, None] * 
     fair_params_df.loc[fair_params_df.index[0], 'solar_amplitude'] + 
     trend_shape[:, None, None] * fair_params_df.loc[fair_params_df.index[0], 'solar_trend'],
     specie='Solar'
)
f.fill_species_configs()
# Climate response
fill(f.climate_configs['ocean_heat_capacity'], fair_params_df.loc[fair_params_df.index[0],'c1':'c3'])
fill(f.climate_configs['ocean_heat_transfer'], fair_params_df.loc[fair_params_df.index[0],'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[fair_params_df.index[0],'epsilon'])
fill(f.climate_configs['sigma_eta'], fair_params_df.loc[fair_params_df.index[0],'sigma_eta'])
fill(f.climate_configs['sigma_xi'], fair_params_df.loc[fair_params_df.index[0],'sigma_xi'])
fill(f.climate_configs['gamma_autocorrelation'], fair_params_df.loc[fair_params_df.index[0],'gamma'])
fill(f.climate_configs['seed'], fair_params_df.index[0]*399 + 1355763)
fill(f.climate_configs['use_seed'], True)
fill(f.climate_configs['stochastic_run'], False)

# carbon cycle
fill(f.species_configs['iirf_0'], fair_params_df.loc[fair_params_df.index[0], 'r0'], specie='CO2')
fill(f.species_configs['iirf_airborne'], fair_params_df.loc[fair_params_df.index[0], 'rA'], specie='CO2')
fill(f.species_configs['iirf_uptake'], fair_params_df.loc[fair_params_df.index[0], 'rU'], specie='CO2')
fill(f.species_configs['iirf_temperature'], fair_params_df.loc[fair_params_df.index[0], 'rT'], specie='CO2')

# aerosol direct
for specie in ['BC', 'CH4', 'N2O', 'NH3', 'NOx', 'OC', 'Sulfur', 'VOC', 'Equivalent effective stratospheric chlorine']:
    fill(f.species_configs['erfari_radiative_efficiency'], fair_params_df.loc[fair_params_df.index[0], f'ari {specie}'], specie=specie)

# aerosol indirect
fill(f.species_configs['aci_parameters'], fair_params_df.loc[fair_params_df.index[0], 'beta'], aci_parameter='scale')
fill(f.species_configs['aci_parameters'], fair_params_df.loc[fair_params_df.index[0], 'shape_so2'], aci_parameter='Sulfur')
fill(f.species_configs['aci_parameters'], fair_params_df.loc[fair_params_df.index[0], 'shape_bcoc'], aci_parameter='BC+OC')

# ozone
for specie in ['CH4', 'N2O', 'Equivalent effective stratospheric chlorine', 'CO', 'VOC', 'NOx']:
    fill(f.species_configs['ozone_radiative_efficiency'], fair_params_df.loc[fair_params_df.index[0], f'o3 {specie}'], specie=specie)

# methane lifetime baseline
fill(f.species_configs['unperturbed_lifetime'], 10.4198121, specie='CH4')

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people might wanna run pulse expts with these gases)
fill(f.species_configs['baseline_emissions'], 19.019783117809567, specie='CH4')
fill(f.species_configs['baseline_emissions'], 0.08602230754, specie='N2O')

# forcing scaling

for specie in ['CH4', 'N2O', 'Stratospheric water vapour', 'Contrails', 'Light absorbing particles on snow and ice', 'Land use']:
    fill(f.species_configs['forcing_scale'], fair_params_df.loc[fair_params_df.index[0], f'scale {specie}'], specie=specie)
for specie in ['CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18', 'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
    'HFC-365mfc', 'HFC-4310mee']:
    fill(f.species_configs['forcing_scale'], fair_params_df.loc[fair_params_df.index[0], 'scale minorGHG'], specie=specie)

calibrated_f4co2_mean = 7.866801427264765
fill(f.species_configs['forcing_scale'], 1 + 0.561*(calibrated_f4co2_mean - fair_params_df.loc[fair_params_df.index[0],'F_4xCO2'])/calibrated_f4co2_mean, specie='CO2')


# tune down volcanic efficacy
fill(f.species_configs['forcing_efficacy'], 0.6, specie='Volcanic')


# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(f.species_configs['baseline_concentration'], fair_params_df.loc[fair_params_df.index[0], 'co2_concentration_1750'], specie='CO2')

f.ch4_method='Thornhill2021'

initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.airborne_emissions, 0)
initialise(f.cumulative_emissions, 0)

f.run()

In [ ]:
f1 = FAIR()

In [ ]:
f1.define_time(1750, 2022, 1)

In [ ]:
f1.define_scenarios(['ssp126'])

In [ ]:
species, properties = read_properties()

In [ ]:
f1.define_species(species, properties)

In [ ]:
[fair_params_df.index[0]]

In [ ]:
f1.define_configs([fair_params_df.index[0]])

In [ ]:
f1.allocate()

In [ ]:
f1.fill_from_rcmip()

In [ ]:
# Volcanic forcing
fill(f1.forcing, volcanic_forcing[:273, None, None] * fair_params_df.loc[fair_params_df.index[0], 'scale Volcanic'], specie='Volcanic')

# Solar forcing
fill(f1.forcing, 
     solar_forcing[:273, None, None] * 
     fair_params_df.loc[fair_params_df.index[0], 'solar_amplitude'] + 
     trend_shape[:273, None, None] * fair_params_df.loc[fair_params_df.index[0], 'solar_trend'],
     specie='Solar'
)

In [ ]:
f1.fill_species_configs()

In [ ]:
# Climate response
fill(f1.climate_configs['ocean_heat_capacity'], fair_params_df.loc[fair_params_df.index[0],'c1':'c3'])
fill(f1.climate_configs['ocean_heat_transfer'], fair_params_df.loc[fair_params_df.index[0],'kappa1':'kappa3'])
fill(f1.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[fair_params_df.index[0],'epsilon'])
fill(f1.climate_configs['sigma_eta'], fair_params_df.loc[fair_params_df.index[0],'sigma_eta'])
fill(f1.climate_configs['sigma_xi'], fair_params_df.loc[fair_params_df.index[0],'sigma_xi'])
fill(f1.climate_configs['gamma_autocorrelation'], fair_params_df.loc[fair_params_df.index[0],'gamma'])
fill(f1.climate_configs['seed'], fair_params_df.index[0]*399 + 1355763)
fill(f1.climate_configs['use_seed'], True)
fill(f1.climate_configs['stochastic_run'], False)

# carbon cycle
fill(f1.species_configs['iirf_0'], fair_params_df.loc[fair_params_df.index[0], 'r0'], specie='CO2')
fill(f1.species_configs['iirf_airborne'], fair_params_df.loc[fair_params_df.index[0], 'rA'], specie='CO2')
fill(f1.species_configs['iirf_uptake'], fair_params_df.loc[fair_params_df.index[0], 'rU'], specie='CO2')
fill(f1.species_configs['iirf_temperature'], fair_params_df.loc[fair_params_df.index[0], 'rT'], specie='CO2')

# aerosol direct
for specie in ['BC', 'CH4', 'N2O', 'NH3', 'NOx', 'OC', 'Sulfur', 'VOC', 'Equivalent effective stratospheric chlorine']:
    fill(f1.species_configs['erfari_radiative_efficiency'], fair_params_df.loc[fair_params_df.index[0], f'ari {specie}'], specie=specie)

# aerosol indirect
fill(f1.species_configs['aci_parameters'], fair_params_df.loc[fair_params_df.index[0], 'beta'], aci_parameter='scale')
fill(f1.species_configs['aci_parameters'], fair_params_df.loc[fair_params_df.index[0], 'shape_so2'], aci_parameter='Sulfur')
fill(f1.species_configs['aci_parameters'], fair_params_df.loc[fair_params_df.index[0], 'shape_bcoc'], aci_parameter='BC+OC')

# ozone
for specie in ['CH4', 'N2O', 'Equivalent effective stratospheric chlorine', 'CO', 'VOC', 'NOx']:
    fill(f1.species_configs['ozone_radiative_efficiency'], fair_params_df.loc[fair_params_df.index[0], f'o3 {specie}'], specie=specie)

# methane lifetime baseline
fill(f1.species_configs['unperturbed_lifetime'], 10.4198121, specie='CH4')

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people might wanna run pulse expts with these gases)
fill(f1.species_configs['baseline_emissions'], 19.019783117809567, specie='CH4')
fill(f1.species_configs['baseline_emissions'], 0.08602230754, specie='N2O')

# forcing scaling

for specie in ['CH4', 'N2O', 'Stratospheric water vapour', 'Contrails', 'Light absorbing particles on snow and ice', 'Land use']:
    fill(f1.species_configs['forcing_scale'], fair_params_df.loc[fair_params_df.index[0], f'scale {specie}'], specie=specie)
for specie in ['CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18', 'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
    'HFC-365mfc', 'HFC-4310mee']:
    fill(f1.species_configs['forcing_scale'], fair_params_df.loc[fair_params_df.index[0], 'scale minorGHG'], specie=specie)

calibrated_f4co2_mean = 7.866801427264765
fill(f1.species_configs['forcing_scale'], 1 + 0.561*(calibrated_f4co2_mean - fair_params_df.loc[fair_params_df.index[0],'F_4xCO2'])/calibrated_f4co2_mean, specie='CO2')


# tune down volcanic efficacy
fill(f1.species_configs['forcing_efficacy'], 0.6, specie='Volcanic')


# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(f1.species_configs['baseline_concentration'], fair_params_df.loc[fair_params_df.index[0], 'co2_concentration_1750'], specie='CO2')

f1.ch4_method='Thornhill2021'

In [ ]:
initialise(f1.concentration, f1.species_configs['baseline_concentration'])
initialise(f1.forcing, 0)
initialise(f1.temperature, 0)
initialise(f1.airborne_emissions, 0)
initialise(f1.cumulative_emissions, 0)

In [ ]:
f1.run()

In [ ]:
f1.gas_partitions[0, 0, 2, :]

In [ ]:
f2 = FAIR()

In [ ]:
f2.define_time(2022, 2100, 1)

In [ ]:
f2.define_scenarios(['ssp126'])

In [ ]:
species, properties = read_properties()

In [ ]:
f2.define_species(species, properties)

In [ ]:
[fair_params_df.index[0]]

In [ ]:
f2.define_configs([fair_params_df.index[0]])

In [ ]:
f2.allocate()

In [ ]:
f2.fill_from_rcmip()

In [ ]:
# Volcanic forcing
fill(f2.forcing, volcanic_forcing[272:, None, None] * fair_params_df.loc[fair_params_df.index[0], 'scale Volcanic'], specie='Volcanic')

# Solar forcing
fill(f2.forcing, 
     solar_forcing[272:, None, None] * 
     fair_params_df.loc[fair_params_df.index[0], 'solar_amplitude'] + 
     trend_shape[272:, None, None] * fair_params_df.loc[fair_params_df.index[0], 'solar_trend'],
     specie='Solar'
)

In [ ]:
f2.fill_species_configs()

In [ ]:
# Climate response
fill(f2.climate_configs['ocean_heat_capacity'], fair_params_df.loc[fair_params_df.index[0],'c1':'c3'])
fill(f2.climate_configs['ocean_heat_transfer'], fair_params_df.loc[fair_params_df.index[0],'kappa1':'kappa3'])
fill(f2.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[fair_params_df.index[0],'epsilon'])
fill(f2.climate_configs['sigma_eta'], fair_params_df.loc[fair_params_df.index[0],'sigma_eta'])
fill(f2.climate_configs['sigma_xi'], fair_params_df.loc[fair_params_df.index[0],'sigma_xi'])
fill(f2.climate_configs['gamma_autocorrelation'], fair_params_df.loc[fair_params_df.index[0],'gamma'])
fill(f2.climate_configs['seed'], fair_params_df.index[0]*399 + 1355763)
fill(f2.climate_configs['use_seed'], True)
fill(f2.climate_configs['stochastic_run'], False)

# carbon cycle
fill(f2.species_configs['iirf_0'], fair_params_df.loc[fair_params_df.index[0], 'r0'], specie='CO2')
fill(f2.species_configs['iirf_airborne'], fair_params_df.loc[fair_params_df.index[0], 'rA'], specie='CO2')
fill(f2.species_configs['iirf_uptake'], fair_params_df.loc[fair_params_df.index[0], 'rU'], specie='CO2')
fill(f2.species_configs['iirf_temperature'], fair_params_df.loc[fair_params_df.index[0], 'rT'], specie='CO2')

# aerosol direct
for specie in ['BC', 'CH4', 'N2O', 'NH3', 'NOx', 'OC', 'Sulfur', 'VOC', 'Equivalent effective stratospheric chlorine']:
    fill(f2.species_configs['erfari_radiative_efficiency'], fair_params_df.loc[fair_params_df.index[0], f'ari {specie}'], specie=specie)

# aerosol indirect
fill(f2.species_configs['aci_parameters'], fair_params_df.loc[fair_params_df.index[0], 'beta'], aci_parameter='scale')
fill(f2.species_configs['aci_parameters'], fair_params_df.loc[fair_params_df.index[0], 'shape_so2'], aci_parameter='Sulfur')
fill(f2.species_configs['aci_parameters'], fair_params_df.loc[fair_params_df.index[0], 'shape_bcoc'], aci_parameter='BC+OC')

# ozone
for specie in ['CH4', 'N2O', 'Equivalent effective stratospheric chlorine', 'CO', 'VOC', 'NOx']:
    fill(f2.species_configs['ozone_radiative_efficiency'], fair_params_df.loc[fair_params_df.index[0], f'o3 {specie}'], specie=specie)

# methane lifetime baseline
fill(f2.species_configs['unperturbed_lifetime'], 10.4198121, specie='CH4')

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people might wanna run pulse expts with these gases)
fill(f2.species_configs['baseline_emissions'], 19.019783117809567, specie='CH4')
fill(f2.species_configs['baseline_emissions'], 0.08602230754, specie='N2O')

# forcing scaling

for specie in ['CH4', 'N2O', 'Stratospheric water vapour', 'Contrails', 'Light absorbing particles on snow and ice', 'Land use']:
    fill(f2.species_configs['forcing_scale'], fair_params_df.loc[fair_params_df.index[0], f'scale {specie}'], specie=specie)
for specie in ['CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18', 'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
    'HFC-365mfc', 'HFC-4310mee']:
    fill(f2.species_configs['forcing_scale'], fair_params_df.loc[fair_params_df.index[0], 'scale minorGHG'], specie=specie)

calibrated_f4co2_mean = 7.866801427264765
fill(f2.species_configs['forcing_scale'], 1 + 0.561*(calibrated_f4co2_mean - fair_params_df.loc[fair_params_df.index[0],'F_4xCO2'])/calibrated_f4co2_mean, specie='CO2')


# tune down volcanic efficacy
fill(f2.species_configs['forcing_efficacy'], 0.6, specie='Volcanic')


# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(f2.species_configs['baseline_concentration'], fair_params_df.loc[fair_params_df.index[0], 'co2_concentration_1750'], specie='CO2')

f2.ch4_method='Thornhill2021'

In [ ]:
f1.concentration[-1, ...]

In [ ]:
initialise(f2.concentration, f1.concentration[-1, ...])
initialise(f2.forcing, f1.forcing[-1, ...])
initialise(f2.temperature, f1.temperature[-1, ...])
initialise(f2.airborne_emissions, f1.airborne_emissions[-1, ...])
initialise(f2.cumulative_emissions, f1.cumulative_emissions[-1, ...])
f2.gas_partitions=f1.gas_partitions

In [ ]:
f2.run()

In [ ]:
pl.plot(f.timebounds, f.temperature[:,0,0,0]-f.temperature[100:151,0,0,0].mean())
pl.plot(f1.timebounds, f1.temperature[:,0,0,0]-f1.temperature[100:151,0,0,0].mean())
pl.plot(f2.timebounds, f2.temperature[:,0,0,0]-f1.temperature[100:151,0,0,0].mean())